In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans

In [2]:
# load the datasets
noaa_ind = pd.read_csv('Data/NOAA_CLN.csv')
usgs_ma35 = pd.read_csv('Data/USGS_MA35.csv')
usgs_ma39 = pd.read_csv('Data/USGS_MA39.csv')

In [3]:
# rename date columns for consistency
usgs_ma35.rename(columns={'datetime': 'date'}, inplace=True)
usgs_ma39.rename(columns={'datetime': 'date'}, inplace=True)

# convert date columns to datetime
noaa_ind['date'] = pd.to_datetime(noaa_ind['date'])

usgs_ma35['date'] = pd.to_datetime(usgs_ma35['date']).dt.date
usgs_ma35['date'] = pd.to_datetime(usgs_ma35['date'])

usgs_ma39['date'] = pd.to_datetime(usgs_ma39['date']).dt.date
usgs_ma39['date'] = pd.to_datetime(usgs_ma39['date'])

# merge NOAA data with groundwater data for both USGS stations
merged_ma35 = usgs_ma35.merge(noaa_ind, on='date', how='inner')
merged_ma39 = usgs_ma39.merge(noaa_ind, on='date', how='inner')

In [4]:
# select only temperature and precipitation features for clustering
clustering_features_ma35 = merged_ma35[['t_median', 'prcp']]
clustering_features_ma39 = merged_ma39[['t_median', 'prcp']]

In [5]:
# apply K-Means clustering
kmeans_ma35 = KMeans(n_clusters=2, random_state=30)
merged_ma35['cluster'] = kmeans_ma35.fit_predict(clustering_features_ma35)

kmeans_ma39 = KMeans(n_clusters=2, random_state=30)
merged_ma39['cluster'] = kmeans_ma39.fit_predict(clustering_features_ma39)

C:\Users\ariag\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ariag\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [6]:
# assign NOAA data dynamically based on cluster
merged_ma35['assigned_station'] = merged_ma35['cluster'].apply(lambda x: 'USW00093819' if x == 0 else 'USW00053842')
merged_ma39['assigned_station'] = merged_ma39['cluster'].apply(lambda x: 'USW00093819' if x == 0 else 'USW00053842')

In [7]:
# drop rows where 'id' and 'assigned_station' are different
merged_ma35 = merged_ma35[merged_ma35['id'] == merged_ma35['assigned_station']]
merged_ma39 = merged_ma39[merged_ma39['id'] == merged_ma39['assigned_station']]

In [8]:
merged_ma35['gwl_median'] = (merged_ma35['gwl_max'] + merged_ma35['gwl_min']) / 2
merged_ma35 = merged_ma35.drop(columns=['gwl_max', 'gwl_min', 'id', 't_hr', 'cluster', 'assigned_station'])
merged_ma35.head()

,date,prcp,t_median,gwl_median
0,2015-01-01,0.00,26.0,33.000
2,2015-01-02,0.00,32.0,32.925
4,2015-01-03,0.79,42.5,32.910
6,2015-01-04,0.01,28.0,32.850
8,2015-01-05,0.11,9.0,32.935


In [9]:
merged_ma39['gwl_median'] = (merged_ma39['gwl_max'] + merged_ma39['gwl_min']) / 2
merged_ma39 = merged_ma39.drop(columns=['gwl_max', 'gwl_min', 'id', 't_hr', 'cluster', 'assigned_station'])
merged_ma39.head()

,date,prcp,t_median,gwl_median
1,2015-01-01,0.00,27.0,17.335
3,2015-01-02,0.00,32.5,17.455
5,2015-01-03,0.81,42.5,17.235
7,2015-01-04,0.01,29.0,16.710
9,2015-01-05,0.10,10.0,16.105


In [10]:
merged_ma35.to_csv('Data/Dynamic_MA35.csv', index=False)
merged_ma39.to_csv('Data/Dynamic_MA39.csv', index=False)